In [1]:
from notebooks.rapprochements.closest import find_closest_building
import csv
from pprint import pprint
import json

## 1 Creation d'un identifiant unique par ligne

In [9]:
with open("bat_rnb_mesr.csv", "r") as f, open("data/bat_rnb_mesr_with_id.csv", "w") as f_output:
    reader = csv.DictReader(f, delimiter=";")
    
    rapprochement_id = 1
    rows_output = []
    
    for row in reader:
        row["rapprochement_id"] = rapprochement_id
        rapprochement_id += 1
        rows_output.append(row)
    
    writer = csv.DictWriter(f_output, fieldnames=rows_output[0].keys() , delimiter=";")
    writer.writeheader()
    writer.writerows(rows_output)
        
        

## 2 Rapprochement par closest building

In [19]:
with open("data/bat_rnb_mesr_with_id.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=";")
    results = {}
    
    
    c = 0
    for row in reader:
        
        c += 1
        
        # print(c)
        
        id_es = row["rapprochement_id"]
        lat = row["lat"]
        lon = row["long"]
            
        url, closest_bdgs = find_closest_building(lat, lon, 20)            
        
        results[id_es] = {
            'query': url,
            'input': row,
            'result': closest_bdgs
        }
        
    with open('results/results-closest.json', 'w') as f:
        json.dump(results, f)
        
    

## 3 Rapprochement par guess

In [15]:
import csv
import json
from pprint import pprint
from notebooks.rapprochements.guess import guess_all

def guess():
    with open("data/bat_rnb_mesr_with_id.csv", "r") as f:
        reader = csv.DictReader(f, delimiter=";")
        all = []
        
        for i, line in enumerate(list(reader)):

            # Address
            address = [
                line["Adresse"],
                line["CP"],
                line["Ville"]
            ]
            address = [x for x in address if x != ""]

            # Point
            point = None
            if line["lat"] and line["long"]:
                point = f"{line['lat']},{line['long']}"

            all.append({
                "ext_id": line["rapprochement_id"],
                "name": line["Libellé bât/ter"],
                "address": " ".join(address),
                "point": point,
            })

        results = guess_all(all, avoid_throttling=True)
        json.dump(results, open("results/results_guess.json", "w"), indent=4)

In [17]:
guess()

guess all : 6335 lines
guess line : 1/6335
---
1
{'name': "Bâtiment Hôtel à Projet et siège de l'EPCC (HAP-EPCC)", 'address': '8  CRS DES HUMANITES 93300 Aubervilliers', 'point': '48.912758,2.398494'}
guess line : 2/6335
---
2
{'name': 'Bâtiment Centre de Colloques (COLL)', 'address': '2  R DES FILLETTES 93300 Aubervilliers', 'point': '48.90752,2.365921'}
guess line : 3/6335
---
3
{'name': 'Espace Associatif et culturel (EAC)', 'address': '15  CRS DES HUMANITES 93300 Aubervilliers', 'point': '48.913479,2.398616'}
guess line : 4/6335
---
4
{'name': "Bâtiment de Recherche de l'EHESS", 'address': '2  CRS DES HUMANITES 93300 Aubervilliers', 'point': '48.912059,2.398257'}
guess line : 5/6335
---
5
{'name': 'Bâtiment Grand Equipement Documentaire (GED)', 'address': '10  CRS DES HUMANITES 93300 Aubervilliers', 'point': '48.913144,2.398598'}
guess line : 6/6335
---
6
{'name': "Siège de l'Institut National des Etudes Démographiques(INED)", 'address': '9  CRS DES HUMANITES 93300 Aubervilliers', 

guess line : 52/6335
---
52
{'name': 'M207-02-GRAND AMPHITHEATRE', 'address': '3  PL VICTOR HUGO 13003 Marseille  3e', 'point': '43.305171,5.378031'}
guess line : 53/6335
---
53
{'name': 'M207-03-SALLE DE CONFERENCES', 'address': '3  PL VICTOR HUGO 13003 Marseille  3e', 'point': '43.305535,5.378104'}
guess line : 54/6335
---
54
{'name': 'BATIMENT LA CANEBIERE -M2015-01', 'address': '115  VOIE COMMUNALE LA CANEBIERE 13001 Marseille  1er', 'point': '43.298434,5.382518'}
guess line : 55/6335
---
55
{'name': 'BATIMENT FAC DE DROIT-M204-01', 'address': '110  VOIE COMMUNALE LA CANEBIERE 13001 Marseille  1er', 'point': '43.297984,5.382735'}
guess line : 56/6335
---
56
{'name': 'M207-05-BATIMENT 5', 'address': '3  PL VICTOR HUGO 13003 Marseille  3e', 'point': '43.305791,5.377583'}
guess line : 57/6335
---
57
{'name': 'M210-07-FACULTE  D ONDONTOLOGIE', 'address': '27  BD JEAN MOULIN 13005 Marseille  5e', 'point': '43.286925,5.402755'}
guess line : 58/6335
---
58
{'name': 'I.F.M.K', 'address': '

---
105
{'name': 'BATIMENT STAPS ADMINISTRATION RECHERCHE+ENSEIGNEMENT M301-18', 'address': '163  AV DE LUMINY 13009 Marseille  9e', 'point': '43.232823,5.445129'}
guess line : 106/6335
---
106
{'name': 'M301-20 COSEC', 'address': '163  AV DE LUMINY 13009 Marseille  9e', 'point': '43.228662,5.441161'}
guess line : 107/6335
---
107
{'name': 'M301-21-HALLE DES SPORTS', 'address': '163  AV DE LUMINY 13009 Marseille  9e', 'point': '43.228441,5.442699'}
guess line : 108/6335
---
108
{'name': 'M301-22  KIR', 'address': '163  AV DE LUMINY 13009 Marseille  9e', 'point': '43.233532,5.44488'}
guess line : 109/6335
---
109
{'name': 'M301-23  IOA / IRPHE', 'address': '163  AV DE LUMINY 13009 Marseille  9e', 'point': '43.235326,5.439786'}
guess line : 110/6335
---
110
{'name': 'M301-24-SOUFFLERIE', 'address': '163  AV DE LUMINY 13009 Marseille  9e', 'point': '43.234014,5.443901'}
guess line : 111/6335
---
111
{'name': 'M301-25  IUT GENIE TELECOMMUNICATION RESEAUX', 'address': '163  AV DE LUMINY 130

guess line : 157/6335
---
157
{'name': 'BATIMENT 101-04 TP CHIMIE-RIVALS-PREFA D', 'address': 'AV ESCADRILLE NORMANDIE NIEMEN 13013 Marseille 13e', 'point': '43.336225,5.412585'}
guess line : 158/6335
---
158
{'name': 'BATIMENT M101-06 PREFA SALLE F-G', 'address': 'AV ESCADRILLE NORMANDIE NIEMEN 13013 Marseille 13e', 'point': '43.337509,5.41352'}
guess line : 159/6335
---
159
{'name': 'BATIMENT M101-07 PREFA-SALLE H', 'address': 'AV ESCADRILLE NORMANDIE NIEMEN 13013 Marseille 13e', 'point': '43.337498,5.413897'}
guess line : 160/6335
---
160
{'name': 'BATIMENT M101-08 PREFA-SALLE E1-E2-E3', 'address': 'AV ESCADRILLE NORMANDIE NIEMEN 13013 Marseille 13e', 'point': '43.337762,5.413545'}
guess line : 161/6335
---
161
{'name': 'BATIMENT M101-09 ATELIER FRESNEL', 'address': 'AV ESCADRILLE NORMANDIE NIEMEN 13013 Marseille 13e', 'point': '43.339449,5.412423'}
guess line : 162/6335
---
162
{'name': 'BATIMENT M101-10 TUBE A CHOCS', 'address': 'AV ESCADRILLE NORMANDIE NIEMEN 13013 Marseille 13e'

---
207
{'name': 'BATIMENT E-IUFM', 'address': '8  AV JULES ISAAC 13100 Aix-en-Provence', 'point': '43.533755,5.451628'}
guess line : 208/6335
---
208
{'name': 'BATIMENT F-IUFM', 'address': '8  AV JULES ISAAC 13100 Aix-en-Provence', 'point': '43.533751,5.452365'}
guess line : 209/6335
---
209
{'name': 'BATIMENT G -IUFM', 'address': '8  AV JULES ISAAC 13100 Aix-en-Provence', 'point': '43.533944,5.4527'}
guess line : 210/6335
---
210
{'name': 'BATIMENT I -DERRIERE SALLE DE CONFERENCE -IUFM', 'address': '8  AV JULES ISAAC 13100 Aix-en-Provence', 'point': '43.534088,5.451575'}
guess line : 211/6335
---
211
{'name': 'BATIMENT SOUS STATION CAFETERIA-IUFM', 'address': '8  AV JULES ISAAC 13100 Aix-en-Provence', 'point': '43.533431,5.451391'}
guess line : 212/6335
---
212
{'name': 'BATIMENT PREFABRIQUE', 'address': '140  RTE DE TARASCON 84140 Avignon', 'point': '43.9264,4.808222'}
guess line : 213/6335
---
213
{'name': 'ESPE (ex-IUFM)-VILLA B', 'address': '15  AV JOSEPH REINACH 04000 Digne-les-

...

guess line : 6093/6335
---
6093
{'name': 'BAT 105', 'address': 'RUE JEAN DOMINIQUE CASSINI 91440 Bures-sur-Yvette', 'point': '48.69843,2.172841'}
guess line : 6094/6335
---
6094
{'name': 'BAT 106', 'address': 'RUE HENRI BECQUEREL 91440 Bures-sur-Yvette', 'point': '48.69705648,2.174332738'}
guess line : 6095/6335
---
6095
{'name': 'BAT 107', 'address': 'RUE HENRI BECQUEREL 91400 Orsay', 'point': '48.697124,2.176127'}
guess line : 6096/6335
---
6096
{'name': 'BAT 199', 'address': 'RUE DE LA CHAUFFERIE 91440 Bures-sur-Yvette', 'point': '48.697141,2.175288'}
guess line : 6097/6335
---
6097
{'name': 'BAT 200', 'address': 'RUE ANDRE AMPERE 91440 Bures-sur-Yvette', 'point': '48.699138,2.169837'}
guess line : 6098/6335
---
6098
{'name': 'BAT 201', 'address': 'RUE ANDRE AMPERE 91440 Bures-sur-Yvette', 'point': '48.698919,48.698919'}
guess line : 6099/6335
---
6099
{'name': 'BAT 202', 'address': 'RUE HENRI BECQUEREL 91440 Bures-sur-Yvette', 'point': '48.698629,2.170535'}
guess line : 6100/6335
-

guess line : 6154/6335
---
6154
{'name': 'BAT 510 ATELIER', 'address': '20  R ANDRE RIVIERE 91400 Orsay', 'point': '48.707522,2.178903'}
guess line : 6155/6335
---
6155
{'name': 'BAT 598', 'address': '15  RUE ANDRE RIVIERE 91400 Orsay', 'point': '48.707098,2.178321'}
guess line : 6156/6335
---
6156
{'name': 'BAT 599', 'address': '15  RUE ANDRE RIVIERE 91400 Orsay', 'point': '48.707331,2.178206'}
guess line : 6157/6335
---
6157
{'name': 'BAT 620', 'address': '18  R JOLIOT CURIE 91190 Gif-sur-Yvette', 'point': '48.709094,2.171468'}
guess line : 6158/6335
---
6158
{'name': 'BATIMENT A', 'address': '5  R JEAN-BAPTISTE CLEMENT 92290 Châtenay-Malabry', 'point': '48.761923,2.272266'}
guess line : 6159/6335
---
6159
{'name': 'BATIMENT B', 'address': '5  R JEAN-BAPTISTE CLEMENT 92290 Châtenay-Malabry', 'point': '48.762673,2.272459'}
guess line : 6160/6335
---
6160
{'name': 'BATIMENT D', 'address': '5  R JEAN-BAPTISTE CLEMENT 92290 Châtenay-Malabry', 'point': '48.76193,2.274358'}
guess line : 61

guess line : 6212/6335
---
6212
{'name': 'BAT 610 J', 'address': '91190 Gif-sur-Yvette', 'point': '48.710184,2.169419'}
guess line : 6213/6335
---
6213
{'name': 'ATELIER 611', 'address': 'rue noetzlin 91190 Gif-sur-Yvette', 'point': '48.710467,2.171382'}
guess line : 6214/6335
---
6214
{'name': 'BAT 612 HTBT/EDF', 'address': 'rue noetzlin 91190 Gif-sur-Yvette', 'point': '48.709831,2.171068'}
guess line : 6215/6335
---
6215
{'name': 'BAT 97', 'address': 'RUE JEAN TEILLAC 91400 Orsay', 'point': '48.699447,2.178471'}
guess line : 6216/6335
---
6216
{'name': 'BAT 98', 'address': 'RUE JEAN TEILLAC 91400 Orsay', 'point': '48.699223,2.177787'}
guess line : 6217/6335
---
6217
{'name': 'BAT 99', 'address': 'AVENUE IRENE ET JOLIOT CURIE 91400 Orsay', 'point': '48.698742,2.178678'}
guess line : 6218/6335
---
6218
{'name': 'BAT 100', 'address': 'av irene et joliot curie 91400 Orsay', 'point': '48.69833821,2.176789641'}
guess line : 6219/6335
---
6219
{'name': 'BAT 101', 'address': 'AV IRENE ET JOL

guess line : 6273/6335
---
6273
{'name': 'BAT 425/428', 'address': 'RUE DU DOYEN GEORGES POITOU 91400 Orsay', 'point': '48.703521,2.177154'}
guess line : 6274/6335
---
6274
{'name': 'BAT 430/433', 'address': 'RUE DU DOYEN GEORGES POITOU 91400 Orsay', 'point': '48.703068,2.175556'}
guess line : 6275/6335
---
6275
{'name': 'BAT 440/447', 'address': 'RUE DU DOYEN GEORGES POITOU 91400 Orsay', 'point': '48.703366,2.17886'}
guess line : 6276/6335
---
6276
{'name': 'BAT 450', 'address': 'RUE CLAUDE BERNARD 91400 Orsay', 'point': '48.703259,2.180631'}
guess line : 6277/6335
---
6277
{'name': 'BAT 452', 'address': 'RUE DU CHATEAU 91400 Orsay', 'point': '48.702304,2.180523'}
guess line : 6278/6335
---
6278
{'name': 'BAT 490/497', 'address': 'RUE HECTOR BERLIOZ 91400 Orsay', 'point': '48.70202,2.181907'}
guess line : 6279/6335
---
6279
{'name': 'BAT 500', 'address': 'ROUTE DU PLATEAU 91400 Orsay', 'point': '48.705635,2.183855'}
guess line : 6280/6335
---
6280
{'name': 'BAT 504', 'address': 'rue d

---
6329
{'name': 'Bâtiment PREMIERS CYCLES', 'address': 'LD LA MARE A LA BESACE 91000 Évry-Courcouronnes', 'point': '48.627616,2.446913'}
guess line : 6330/6335
---
6330
{'name': 'BIBLIOTHEQUE UNIVERSITAIRE', 'address': '2  R ANDRE LALANDE 91000 Évry-Courcouronnes', 'point': '48.623588,2.425419'}
guess line : 6331/6335
---
6331
{'name': 'DEPARTEMENT GTE', 'address': 'RUE SALVADOR ALLENDE 91220 Brétigny-sur-Orge', 'point': '48.620159,2.3110630000000003'}
guess line : 6332/6335
---
6332
{'name': 'BATIMENT ADMINISTRATIF', 'address': 'AV SALVADOR ALLENDE 91220 Brétigny-sur-Orge', 'point': '48.618608,2.306561'}
guess line : 6333/6335
---
6333
{'name': 'IUT DE JUVISY', 'address': '6  ALEXANDRE PIVER 91260 Juvisy-sur-Orge', 'point': '48.689855,2.373161'}
guess line : 6334/6335
---
6334
{'name': 'BATIMENT IBGBI', 'address': 'LD LA BUTTE AUX BERGERS 91000 Évry-Courcouronnes', 'point': '48.618837,2.441052'}
guess line : 6335/6335
---
6335
{'name': "ECOLE NATIONALE SUPERIEURE D'INFORMATIQUE POUR

   ## 4 Analyse

### Histogramme de la distance entre le batiment trouvé et les coordonnées trouvées

In [5]:
import json
import pandas as pd
# count 0 distance results

on_bdg_count = 0
with open('results/results-closest.json', 'r') as f:
    distances = []

    data = json.load(f).items()
    for key, row in data:
        
        result = row['result']
        
        if isinstance(result, dict):
            distances.append(result.get('distance', None))
        else:
            distances.append(None)
        

    df = pd.DataFrame(distances)
    
    total = len(distances)
    d_zero = (df[0] == 0).sum()
    d_max_one = (df[0] <= 1).sum()
    with_result = (df[0].notnull()).sum()
    
    print('-- Rows')
    print(total)
    
    print('-- Distance = 0')
    print(d_zero)
    print(f"{round(d_zero / total * 100, 2)}%")
    
    print('-- Distance <= 1')
    print(d_max_one)
    print(f"{round(d_max_one / total * 100, 2)}%")
    
    print('-- No results')
    print(total - with_result)
    
    
    print('-- Repartition')
    df.hist(bins=10)
    
    

total lines : 100
on bdg : 83


### Quelques stats sur les batiments identifiés par closest

In [151]:
import pandas as pd
from urllib.parse import urlparse, parse_qs

def get_lat_lon_from_query(query):
    parsed_url = urlparse(query)
    query_params = parse_qs(parsed_url.query)
    lat, lon = query_params['point'][0].split(",")
    lat = float(lat)
    lon = float(lon)
    return lat, lon


def get_data(item):
    lat, lon = get_lat_lon_from_query(item['query'])
    #data = {'lat_input': lat, 'lon_input': lon}
    #if isinstance(item['result'], dict):
    #    data = {**data, **result}
    data = item.get('result', {})
    if data is None:
        data = {}
    data.update({'lat_input': lat, 'lon_input': lon})
    return data

with open('results/results-closest.json', 'r') as f:
    data = json.load(f)
    
    # Extract the IDs and 'result' values from each item
    result_values = [{'id': id, 'data': get_data(item)} for id, item in data.items()]

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(result_values)
    # split the unique column to a multiple columns
    df_closest = pd.json_normalize(df['data'])
    
    # je rajoute une colonne qui donne le nombre de fois que un rnb_id a été identifié
    # ce qui peut donner des indices sur de "mauvais" découpages de bâtiments 
    df_closest["count"] = df_closest.groupby('rnb_id')['rnb_id'].transform('count')
    df_closest

### occurences du même rnb_id dans les résultats

In [152]:
 print(df_closest[["rnb_id", "count"]].groupby("count").count())

       rnb_id
count        
1.0      2844
2.0      2112
3.0       549
4.0       240
5.0        65
6.0       120
7.0        14
8.0        64
10.0       20
13.0       13
15.0       15
18.0       18


### nombre de batiments identifiés avec distance à 0 et unique résultat

In [153]:
filtered_df = df_closest[(expanded_df["count"] == 1.0) & (df_closest["distance"] == 0.0)]
print(filtered_df.count())

rnb_id               2045
distance             2045
status               2045
addresses            2045
ext_ids              2045
lat_input            2045
lon_input            2045
point.type           2045
point.coordinates    2045
count                2045
dtype: int64


### Comparaison des résultats exactes et fuzzy

In [178]:
def first_result(item):
    #print(item)
    if 'result' in item and len(item['result']) > 0:
        #print('result found')
        return item['result'][0]
    else:
        return {}

with open("results/results_guess.json") as f_guess:
    data = json.load(f_guess)
    #print(data["1"])
    result_values = [{'id': id, 'result': first_result(item)} for id, item in data.items()]
    df = pd.DataFrame(result_values)
    df_guess = pd.json_normalize(df['result'])
    

In [179]:
# merge the closest and guess results in a single dataframe
df_merge = df_closest.join(df_guess, how='inner', lsuffix='_closest', rsuffix='_guess')
df_merge

,rnb_id_closest,distance,status_closest,addresses_closest,ext_ids_closest,lat_input,lon_input,point.type_closest,point.coordinates_closest,count,rnb_id_guess,score,status_guess,addresses_guess,ext_ids_guess,sub_scores.ban_id_shared,sub_scores.ban_point_plot_cluster,sub_scores.ban_point_distance,point.type_guess,point.coordinates_guess,sub_scores.point_plot_cluster,sub_scores.point_distance
0,7CMPGRQB8MVE,9.864992,[29382184],"[{'id': '93001_4477_00002_b', 'source': 'bdnb'...","[{'id': 'bdnb-bc-KD5G-D5SY-TTD7', 'source': 'b...",48.912758,2.398494,Point,"[2.398634694763835, 48.91261656622051]",1.0,3KC1HAF4GH49,6.000000,"[{'type': 'constructed', 'happened_at': None, ...","[{'id': '93001_4505_00010', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-WGJP-BBLX-ZYCH', 'source': 'b...",1.0,2.0,3.000000,Point,"[2.366967317502423, 48.90967725225204]",NaN,NaN
1,Y3QETN2FK8AQ,8.721511,[26606534],"[{'id': '93066_4757_00017', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-QTS8-9PCR-UWA4', 'source': 'b...",48.907520,2.365921,Point,"[2.365738883188782, 48.90737341178965]",1.0,ERHYPVTP7E6K,3.216753,"[{'type': 'constructed', 'happened_at': None, ...","[{'id': '93001_3530_00004', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-A5Q2-APBX-DD7B', 'source': 'b...",0.0,2.0,0.149077,Point,"[2.366426388742549, 48.90749042421794]",1.0,0.067676
2,TG6NQ4G9ADRZ,7.711687,[30385030],"[{'id': '93001_4477_00015', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-5BGV-P1X5-W9Q8', 'source': 'b...",48.913479,2.398616,Point,"[2.398487557001666, 48.913529779077194]",1.0,XA4B441KMMB8,3.663349,"[{'type': 'constructed', 'happened_at': None, ...","[{'id': '93001_4505_00015', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-CERE-MX5X-A2EP', 'source': 'b...",1.0,2.0,0.663349,Point,"[2.36645445213984, 48.911980812498214]",NaN,NaN
3,3T18WPVQZBHD,12.808872,[43238087],"[{'id': '93001_8110_00140', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-MF2M-GB8Q-5QU2', 'source': 'b...",48.912059,2.398257,Point,"[2.397989162652009, 48.91201383489387]",1.0,AWCS72W4FA2Z,3.183549,"[{'type': 'constructed', 'happened_at': None, ...","[{'id': '93001_4505_00002', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-XVNZ-L7LN-FNL6', 'source': 'b...",1.0,2.0,0.183549,Point,"[2.367216009812121, 48.90797471640751]",NaN,NaN
4,QXK5ANCYV5NV,14.351345,[42520327],"[{'id': '93001_4477_00009', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-5B44-DV8B-G3Y9', 'source': 'b...",48.913144,2.398598,Point,"[2.398363286339152, 48.913064844566115]",1.0,3KC1HAF4GH49,3.305480,"[{'type': 'constructed', 'happened_at': None, ...","[{'id': '93001_4505_00010', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-WGJP-BBLX-ZYCH', 'source': 'b...",1.0,2.0,0.305480,Point,"[2.366967317502423, 48.90967725225204]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6330,947FRHRPBHYK,8.665975,[44665511],[],"[{'id': 'bdnb-bc-VG2G-878T-6T85', 'source': 'b...",48.620159,2.311063,Point,"[2.310917254500024, 48.62013909303511]",1.0,947FRHRPBHYK,1.115394,"[{'type': 'constructed', 'happened_at': None, ...",[],"[{'id': 'bdnb-bc-VG2G-878T-6T85', 'source': 'b...",NaN,NaN,NaN,Point,"[2.310917254500024, 48.62013909303511]",1.0,0.115394
6331,AA3BBE4GBA5N,0.179100,[19473335],"[{'id': '91103_2695_00001', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-LHQP-1ZQ1-UJ22', 'source': 'b...",48.618608,2.306561,Point,"[2.306515278723096, 48.618524896649234]",1.0,AA3BBE4GBA5N,2.000000,"[{'type': 'constructed', 'happened_at': None, ...","[{'id': '91103_2695_00001', 'source': 'bdnb', ...","[{'id': 'bdnb-bc-LHQP-1ZQ1-UJ22', 'source': 'b...",NaN,NaN,NaN,Point,"[2.306515278723096, 48.618524896649234]",1.0,1.000000
6332,EMD18J8MPS5Q,3.228358,[43945817],"[{'id': '91326_2030_00006_a', 'source': 'bdnb'...","[{'id': 'bdnb-bc-CYRZ-Z3K9-LX52', 'source': 'b...",48.689855,2.373161,Point,"[2.373040591527277, 48.689864718633785]",1.0,EMD18J8MPS5Q,1.309755,"[{'type': 'constructed', 'happened_at': None, ...","[{'id': '91326_2030_00006_a', 'source': 'bdnb'...","[{'id': 'bdnb-bc-CYRZ-Z3K9-LX52', 'source':

In [155]:
# count the number of case where the two approaches match
df_merge[df_merge['rnb_id_closest'] == df_merge['rnb_id_guess']]['rnb_id_closest'].count()

5806

#### Cas où l'input tombe sur un batiment

In [156]:
# count the number of time input point is on the building
df_merge[df_merge['distance'] == 0.0]['rnb_id_closest'].count()

4609

In [157]:
# count the number of time input point is on the building and the two approaches match
df_merge[(df_merge['distance'] == 0.0) & (df_merge['rnb_id_closest'] == df_merge['rnb_id_guess'])]['rnb_id_closest'].count()

4586

In [176]:
# list cases where the input is on the building, but guess said something else
pd.set_option('display.max_columns', None)
df_why_guess_different = df_merge[(df_merge['distance'] == 0.0) & (df_merge['rnb_id_closest'] != df_merge['rnb_id_guess'])]
df_why_guess_different.count()

rnb_id_closest                       23
distance                             23
status_closest                       23
addresses_closest                    23
ext_ids_closest                      23
lat_input                            23
lon_input                            23
point.type_closest                   23
point.coordinates_closest            23
count                                23
rnb_id_guess                         23
score                                23
status_guess                         23
addresses_guess                      23
ext_ids_guess                        23
sub_scores.ban_id_shared             23
sub_scores.ban_point_plot_cluster    23
sub_scores.ban_point_distance        23
point.type_guess                     23
point.coordinates_guess              23
sub_scores.point_plot_cluster         2
sub_scores.point_distance             2
dtype: int64

In [186]:
def diff_geojson(df):
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for i, row in df.iterrows():
        #print(row)
        input_feature = {
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": [row["lon_input"], row["lat_input"]]},
            "properties": {"type": "input", "id": i, **row},
        }

        closest_feature = {
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": row["point.coordinates_closest"]},
            "properties": {"type": "closest", "rnb_id": row["rnb_id_closest"]},
        }

        guess_feature = {
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": row["point.coordinates_guess"]},
            "properties": {"type": "guess", "rnb_id": row["rnb_id_guess"]},
        }

        line_feature_closest = {
            "type": "Feature",
            "geometry": {
            "type": "LineString",
            "coordinates": [
                [row["lon_input"], row["lat_input"]],
                row["point.coordinates_closest"]
            ]},
            "properties": {},

        }

        line_feature_guess = {
            "type": "Feature",
            "geometry": {
            "type": "LineString",
            "coordinates": [
                [row["lon_input"], row["lat_input"]],
                row["point.coordinates_guess"]
            ]},
            "properties": {},
        }

        geojson["features"].append(input_feature)
        geojson["features"].append(closest_feature)    
        geojson["features"].append(guess_feature)    
        geojson["features"].append(line_feature_closest)    
        geojson["features"].append(line_feature_guess)    


    return json.dumps(geojson).replace("NaN", '"NaN"').replace("\'", "'")

diff_geojson(df_why_guess_different)

'{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Point", "coordinates": [55.383025, -21.079457]}, "properties": {"type": "input", "id": 1902, "rnb_id_closest": "Y4BW8SBNT3FC", "distance": 0.0, "status_closest": [46814453], "addresses_closest": [], "ext_ids_closest": [{"id": "BATIMENT0000001402336818", "source": "bdtopo", "created_at": "2023-12-22T08:30:55.821448+00:00", "source_version": "bdtopo_2023_09"}], "lat_input": -21.079457, "lon_input": 55.383025, "point.type_closest": "Point", "point.coordinates_closest": [55.38312578417283, -21.079664589902173], "count": 1.0, "rnb_id_guess": "NQ76HPQSXNRK", "score": 0.17907618918688456, "status_guess": [{"type": "constructed", "happened_at": null, "label": "Construit", "is_current": true}], "addresses_guess": [], "ext_ids_guess": [{"id": "BATIMENT0000001401590023", "source": "bdtopo", "created_at": "2023-12-22T08:34:35.021249+00:00", "source_version": "bdtopo_2023_09"}], "sub_scores.ban_id_shared": 0.0, "s

#### différence guess / closest cas général (l'input tombe sur un batiment ou pas)

In [194]:
# number of cases where closest gave no results
df_merge[(df_merge['rnb_id_closest'] != df_merge['rnb_id_guess']) & (df_merge['rnb_id_closest'].isna())]['lat_input'].count()

261

In [195]:
# list cases where the input is on the building, but guess said something else
df_why_guess_different_all = df_merge[(df_merge['rnb_id_closest'] != df_merge['rnb_id_guess']) & (df_merge['rnb_id_closest'].notna()) & (df_merge['rnb_id_guess'].notna())]
df_why_guess_different_all

diff_geojson(df_why_guess_different_all)

'{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Point", "coordinates": [2.398494, 48.912758]}, "properties": {"type": "input", "id": 0, "rnb_id_closest": "7CMPGRQB8MVE", "distance": 9.86499159, "status_closest": [29382184], "addresses_closest": [{"id": "93001_4477_00002_b", "source": "bdnb", "street_number": "2", "street_rep": "B", "street_name": "henri manigart", "street_type": "rue", "city_name": "Aubervilliers", "city_zipcode": "93300", "city_insee_code": "93001"}, {"id": "93001_4477_00002_c", "source": "bdnb", "street_number": "2", "street_rep": "C", "street_name": "henri manigart", "street_type": "rue", "city_name": "Aubervilliers", "city_zipcode": "93300", "city_insee_code": "93001"}, {"id": "93001_4477_00002", "source": "bdnb", "street_number": "2", "street_rep": "", "street_name": "henri manigart", "street_type": "rue", "city_name": "Aubervilliers", "city_zipcode": "93300", "city_insee_code": "93001"}, {"id": "93001_4477_00002_a", "source":